### Install Library

In [1]:
!pip install deepface
!pip install matplotlib
!pip install opencv-python
!pip install fpdf

     |████████████████████████████████| 65 kB 1.6 MB/s 
     |████████████████████████████████| 60.5 MB 134 kB/s 
     |████████████████████████████████| 2.3 MB 52.7 MB/s 
     |████████████████████████████████| 87 kB 7.6 MB/s 
     |████████████████████████████████| 462 kB 72.0 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=3dcd8873dc91f9cdb2e1f4224ed2d9d3c305939a0e25582a9b7b2e83bcaa5d39
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have im

### Import Library

In [2]:
from deepface import DeepFace
from fpdf import FPDF
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd
import os
from pathlib import Path

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


### Salin data dari github ke local drive

In [3]:
!git clone https://github.com/33220311/DeepFace

Cloning into 'DeepFace'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 312 (delta 19), reused 0 (delta 0), pack-reused 262
Receiving objects: 100% (312/312), 65.93 MiB | 57.95 MiB/s, done.
Resolving deltas: 100% (54/54), done.


### Pilihan model, metrics, dan detector yang akan digunakan

In [14]:
detectors = ["opencv", "ssd", "mtcnn", "dlib", "retinaface"]
metrics = ["cosine", "euclidean", "euclidean_l2"]
models = ["VGG-Face", "Facenet", "OpenFace", "DeepFace", "DeepID", "ArcFace", "Dlib", "Facenet512", "SFace"]

### Path dataset testing dan kumpulan foto yang akan menjadi ground truth

In [5]:
source = '/content/DeepFace/Database'
folder ='/content/DeepFace/Test Dataset'

### Fungsi untuk menampilkan seluruh dataset Anggota kelas yang diambil dari folder Anggota kelas

In [6]:
def load_images_from_folder(folder):
    images = []
    labels=[]    
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        #if img is not None:
        images.append(img)
        labels.append(Path(filename).stem)
    return images,labels
x,lbl=load_images_from_folder(folder)
countData=len(x)

### Fungsi untuk mencetak data ke file PDF

*   setiap baris menampilkan nama file pada data test dan nama file yang dianggap mirip dengan data test serta hasil prediksinya (benar atau salah)
*   baris terakhir menampilkan akurasi untuk model, detector, dan metric yang digunakan



In [7]:
def createFile(cetak, filename, accuracy):
  pdf = FPDF()
  pdf.add_page()
  pdf.set_font("Arial", size = 15)
  str1 = "      "
  for i in range(len(cetak)):
    teks = str1.join(cetak[i])
    pdf.cell(200, 10, txt = teks, ln = 1, align = 'L')
  
  pdf.cell(200, 10, txt = "accuracy = "+ accuracy, ln = 1, align = 'L')
  pdf.output(filename)   

### Fungsi untuk menjalankan model tertentu dengan detector dan metric tertentu


*   Pengenalan dihitung benar jika wajah yang ada pada test data dapat dikenali sesuai dengan referensi utama dan wajah yang tidak ada di referensi utama dikategorikan sebagai ‘unknown’.
*   Nilai akurasi dihitung dengan membagi jumlah benar dengan total foto pada test data



In [8]:
def runModel(folder, source, model, metric, detector):
  cetak = []
  true = 0
  false = 0

  for filename in os.listdir(folder):
    name = filename.split("-")

    file = os.path.join(folder,filename)
    df = DeepFace.find(img_path=file, db_path = source, model_name = model, distance_metric = metric, detector_backend = detector, enforce_detection=False)
  
    if df.empty:
      result = "unknown"
    else:
      result = df['identity'][0]
      result = result.split("/")
      result = result[len(result)-1]
      result = result.split(" ")
      result = result[0]

    if result.lower() == name[0].lower():
      true = true + 1
      prediction ="benar"
    else:
      false = false + 1
      prediction = "salah"

    cetak.append([filename,result,prediction])
  
  accuracy = str((true / (true+false)) * 100)
  return cetak, accuracy

### Eksplorasi


*   Mencoba berbagai kombinasi model pengenalan wajah dan deteksi wajah untuk mencari model dengan kinerja terbaik.
*   Hasil prediksi disimpan dalam list kemudian dituliskan ke dalam file PDF
* Kinerja diukur menggunakan metric akurasi
* Kinerja masing-masing model disimpan dalam list


In [9]:
hasil = []

In [16]:
for i in range(len(models)):
  model = models[i]
  for j in range(len(detectors)):
    detector = detectors[j]
    for k in range(len(metrics)):
      metric = metrics[k]

      cetak,acc = runModel(folder, source, model, metric, detector)
      hasil.append([model, metric, detector,acc])
      #outputfile = model+"-"+detector+"-"+metric+".pdf"
      outputfile = model+"-"+detector+"-"+metric+"-"+acc+".pdf"
      createFile(cetak, outputfile, acc)

sface weights will be downloaded...


Downloading...
From: https://github.com/opencv/opencv_zoo/raw/master/models/face_recognition_sface/face_recognition_sface_2021dec.onnx
To: /root/.deepface/weights/face_recognition_sface_2021dec.onnx
100%|██████████| 38.7M/38.7M [00:00<00:00, 176MB/s]


Streaming output truncated to the last 5000 lines.
There are  17  representations found in  representations_sface.pkl
find function lasts  0.042061805725097656  seconds
There are  17  representations found in  representations_sface.pkl
find function lasts  0.16053271293640137  seconds
There are  17  representations found in  representations_sface.pkl
find function lasts  0.08758306503295898  seconds
There are  17  representations found in  representations_sface.pkl
find function lasts  0.22726202011108398  seconds
There are  17  representations found in  representations_sface.pkl
find function lasts  0.09623193740844727  seconds
There are  17  representations found in  representations_sface.pkl
find function lasts  1.18876314163208  seconds
There are  17  representations found in  representations_sface.pkl
find function lasts  0.5352215766906738  seconds
There are  17  representations found in  representations_sface.pkl
find function lasts  0.05649423599243164  seconds
There are  17  r

### Kesimpulan


Akurasi paling baik diperoleh menggunakan model, metric, dan detector berikut:

*  model = 'ArcFace', metric = 'euclidean', detector = 'mtcnn', accuracy = '65.2542372881356'
*  model = 'Facenet512', metric = 'cosine', detector = 'dlib', accuracy = '65.2542372881356'


In [17]:
hasil

[['Dlib', 'cosine', 'opencv', '33.89830508474576'],
 ['Dlib', 'euclidean', 'opencv', '21.1864406779661'],
 ['Dlib', 'euclidean_l2', 'opencv', '25.423728813559322'],
 ['Dlib', 'cosine', 'ssd', '27.11864406779661'],
 ['Dlib', 'euclidean', 'ssd', '13.559322033898304'],
 ['Dlib', 'euclidean_l2', 'ssd', '20.33898305084746'],
 ['Dlib', 'cosine', 'mtcnn', '35.59322033898305'],
 ['Dlib', 'euclidean', 'mtcnn', '22.033898305084744'],
 ['Dlib', 'euclidean_l2', 'mtcnn', '27.966101694915253'],
 ['Dlib', 'cosine', 'dlib', '55.08474576271186'],
 ['Dlib', 'euclidean', 'dlib', '36.440677966101696'],
 ['Dlib', 'euclidean_l2', 'dlib', '46.61016949152542'],
 ['Dlib', 'cosine', 'retinaface', '38.983050847457626'],
 ['Dlib', 'euclidean', 'retinaface', '19.491525423728813'],
 ['Dlib', 'euclidean_l2', 'retinaface', '27.11864406779661'],
 ['Facenet512', 'cosine', 'opencv', '60.16949152542372'],
 ['Facenet512', 'euclidean', 'opencv', '37.28813559322034'],
 ['Facenet512', 'euclidean_l2', 'opencv', '42.3728813559